# PGA Tour Data Retrieval
This notebook is for working with PGA_Data folder for retreiving revelant PGA Tour data that has been webscraped from [pgatour.com](https://www.pgatour.com/).

In [101]:
# General libraries
import pandas as pd
import numpy as np
from importlib import reload
import difflib

# Self made libraries
import pga_data.scrape
import pga_data.mapping

# Reload libraries
reload(pga_data.mapping)
reload(pga_data.scrape)

<module 'pga_data.scrape' from 'C:\\Users\\jborman\\Documents\\Python\\dfs_golf\\pga_data\\scrape.py'>

In [102]:
cat = pga_data.scrape.stat(stat='Top 10 Finishes')
df=cat.pull()

print(cat.stat)
df

{'stat name': 'Top 10 Finishes', 'stat id': '138', 'stat category': 'Money/Finishes', 'years': [1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], 'tournament option': True}


,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,TOP 10,1ST,2ND,3RD
0,1,1,Jon Rahm,12,1.0,1.0,1.0
1,2,2,Jordan Spieth,9,1.0,2.0,2.0
2,T3,T7,Harris English,8,2.0,NaN,1.0
3,T3,T3,Brooks Koepka,8,1.0,2.0,NaN
4,T3,T7,Daniel Berger,8,1.0,NaN,1.0
...,...,...,...,...,...,...,...
175,T125,T124,David Lipsky,1,NaN,NaN,NaN
176,T125,T124,Patrick Rodgers,1,NaN,NaN,NaN
177,T125,T124,Christiaan Bezuidenhout,1,NaN,NaN,NaN
178,T125,T124,Ollie Schniederjans,1,NaN,NaN,1.0


## Try to Extract Tournament IDs from pgatour.com
Webscrape to perform this task

In [161]:
# Attempt all-time records
maps = [pga_data.mapping.shots_gained_map]
cats = ['All-Time Records']

pga_data.mapping.write_stat_meta(maps, cats)

Retrieving All-Time Records...
  SG: Total
  SG: Tee to Green
  SG: Off the Tee
  SG: Approach the Green
  SG: Around the Green
  SG: Putting
...All-Time Records COMPLETE!



## Modify mapping.py functions for new meta data formats

In [97]:
reload(pga_data.mapping)
meta = pga_data.mapping.mapping()

# stat = ['Longest Putts', 'Three-Putts Per Round']
stat = 'Longest Putts'

In [100]:
meta.list_stat('498', from_id=True)['stat id']

'498'